# Linkedin Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [ ]:
!wget https://s3-us-west-2.amazonaws.com/documents.thinknum.com/dataset_dump/flikerqvnk/temp_datalab_records_linkedin_company.zip

In [2]:
file_name= '/Users/adeelahuma/Documents/code_challenge/data/linkedin/temp_datalab_records_linkedin_company.csv'

In [3]:
linkedin_data = pd.read_csv(file_name)

/Users/adeelahuma/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
linkedin_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2426196 entries, 0 to 2426195
Data columns (total 14 columns):
dataset_id               int64
as_of_date               object
company_name             object
followers_count          int64
employees_on_platform    int64
link                     object
industry                 object
date_added               object
date_updated             object
description              object
website                  object
entity_id                float64
cusip                    float64
isin                     float64
dtypes: float64(3), int64(3), object(8)
memory usage: 259.1+ MB


In [5]:
linkedin_data.columns

Index(['dataset_id', 'as_of_date', 'company_name', 'followers_count',
       'employees_on_platform', 'link', 'industry', 'date_added',
       'date_updated', 'description', 'website', 'entity_id', 'cusip', 'isin'],
      dtype='object')

In [6]:
linkedin_data.describe()

,dataset_id,followers_count,employees_on_platform,entity_id,cusip,isin
count,2.426196e+06,2.426196e+06,2.426196e+06,0.0,0.0,0.0
mean,7.347932e+05,7.167738e+04,7.587255e+03,NaN,NaN,NaN
std,5.004440e+05,2.638344e+05,2.412444e+04,NaN,NaN,NaN
min,5.832900e+04,0.000000e+00,0.000000e+00,NaN,NaN,NaN
25%,6.851400e+04,2.148000e+03,2.180000e+02,NaN,NaN,NaN
50%,8.790880e+05,9.335000e+03,1.083000e+03,NaN,NaN,NaN
75%,9.029690e+05,3.864200e+04,4.513000e+03,NaN,NaN,NaN
max,2.339486e+06,7.833967e+06,5.779520e+05,NaN,NaN,NaN


# Data Cleaning
- convert datetime fiels to datetime object
- splitting datetime column to day, year, month, year_month columns
- To calculate employee turn over:
    - shift 'employees_on_platform' by one day (emp_count_shifted_1)
    - subtract 'emp_count_shifted_1' and 'employees_on_platform' to get delta of employees_on_platform per day 
    - fill missing values with 0
    - calculate number of employees left and joined per month and company. sum on positive numbers gives the numbr of employees joined and sum on negative numbers gives the numbers of employee's left 
    - pick the number of employees_on_platform at the start of month for each company 
    - pick the number of employees_on_platform at the end of month for each company 
    - calculate turnover by applying turnover rate formula


##### Turnover rate calculation
- No. of employees at the beginnning  = S
- No. of employees at the end  = E
- Avg = S+E /2
- No. of employee's left = L
- Turnover rate = L/Avg * 100

In [7]:
# lets convert date object to dateTime format
linkedin_data.as_of_date = pd.to_datetime(linkedin_data.as_of_date, 
                                          infer_datetime_format=True)

In [8]:
#lets add column for day, month, year, year-month so that we can see trend by month or year
linkedin_data = linkedin_data.join(linkedin_data.as_of_date.apply(lambda x : pd.Series({
    'day': x.day, 
    'year':x.year, 
    'month': x.month, 
    'year_month': x.to_period('M'),
    'year_month_01': pd.datetime(x.year,x.month,1)
})))

In [9]:
linkedin_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2426196 entries, 0 to 2426195
Data columns (total 19 columns):
dataset_id               int64
as_of_date               datetime64[ns]
company_name             object
followers_count          int64
employees_on_platform    int64
link                     object
industry                 object
date_added               object
date_updated             object
description              object
website                  object
entity_id                float64
cusip                    float64
isin                     float64
day                      int64
year                     int64
month                    int64
year_month               object
year_month_01            datetime64[ns]
dtypes: datetime64[ns](2), float64(3), int64(6), object(8)
memory usage: 351.7+ MB


In [10]:
linkedin_data.as_of_date.describe()

count                 2426196
unique                   1033
top       2018-02-17 00:00:00
freq                     4430
first     2015-09-14 00:00:00
last      2018-07-17 00:00:00
Name: as_of_date, dtype: object

In [11]:
# shift number of employees_on_platform column 1 day to get delta 
linkedin_data['emp_count_shifted_1'] = linkedin_data.groupby(['company_name'])['employees_on_platform'].transform(
    lambda x : x.shift())

# get delta of employees on platform 
linkedin_data['emp_delta'] = linkedin_data.employees_on_platform - linkedin_data.emp_count_shifted_1

linkedin_data.head()

,dataset_id,as_of_date,company_name,followers_count,employees_on_platform,link,industry,date_added,date_updated,description,...,entity_id,cusip,isin,day,year,month,year_month,year_month_01,emp_count_shifted_1,emp_delta
0,58329,2015-09-14,Goldman Sachs,552254,38124,https://www.linkedin.com/company/1382,Investment Banking,2015-09-14 00:00:00+00,2015-09-14 00:00:00+00,NaN,...,NaN,NaN,NaN,14,2015,9,2015-09,2015-09-01,NaN,NaN
1,58329,2015-09-15,Goldman Sachs,552862,38141,https://www.linkedin.com/company/1382,Investment Banking,2015-09-15 00:00:00+00,2015-09-15 00:00:00+00,NaN,...,NaN,NaN,NaN,15,2015,9,2015-09,2015-09-01,38124.0,17.0
2,58363,2015-09-16,United Technologies,59157,14982,https://www.linkedin.com/company/2426,Aviation & Aerospace,2015-09-16 00:00:00+00,2015-09-16 00:00:00+00,NaN,...,NaN,NaN,NaN,16,2015,9,2015-09,2015-09-01,NaN,NaN
3,58366,2015-09-16,Novo Nordisk,336175,26448,https://www.linkedin.com/company/2227,Pharmaceuticals,2015-09-16 00:00:00+00,2015-09-16 00:00:00+00,NaN,...,NaN,NaN,NaN,16,2015,9,2015-09,2015-09-01,NaN,NaN
4,58371,2015-09-16,"Lowe's Companies, Inc.",134255,62574,https://www.linkedin.com/company/4128,Retail,2015-09-16 00:00:00+00,2015-09-16 00:00:00+00,NaN,...,NaN,NaN,NaN,16,2015,9,2015-09,2015-09-01,NaN,NaN


In [12]:
##fill delta column NAN's with 0
linkedin_data.emp_delta.fillna(0.0, inplace=True)

## fill emp_count_shifted_1 column NAN's with 0 
linkedin_data.emp_count_shifted_1.fillna(0, inplace=True)

In [60]:
linkedin_data.head(2)

,dataset_id,as_of_date,company_name,followers_count,employees_on_platform,link,industry,date_added,date_updated,description,...,entity_id,cusip,isin,day,year,month,year_month,year_month_01,emp_count_shifted_1,emp_delta
0,58329,2015-09-14,Goldman Sachs,552254,38124,https://www.linkedin.com/company/1382,Investment Banking,2015-09-14 00:00:00+00,2015-09-14 00:00:00+00,NaN,...,NaN,NaN,NaN,14,2015,9,2015-09,2015-09-01,0.0,0.0
1,58329,2015-09-15,Goldman Sachs,552862,38141,https://www.linkedin.com/company/1382,Investment Banking,2015-09-15 00:00:00+00,2015-09-15 00:00:00+00,NaN,...,NaN,NaN,NaN,15,2015,9,2015-09,2015-09-01,38124.0,17.0


In [61]:
## now calculate the number of employees left and joined per month 

emp_join_left = linkedin_data.groupby(['year_month_01', 'company_name'])['emp_delta'].agg(
    [('left_per_month', lambda x: np.sum(np.abs(x[x<0]))), 
    ('joined_per_month', lambda x: x[x>0].sum())])

emp_join_left.head()


left_per_month  joined_per_month
year_month_01 company_name                                  
2015-09-01    3M                    5350.0             245.0
              58.Com Inc               0.0              38.0
              ADP                   5793.0             289.0
              AIG                      0.0               0.0
              AMD                      0.0               0.0

In [62]:
# merge with main table
linkedin_data = pd.merge(linkedin_data, emp_join_left, on=['year_month_01', 'company_name'])
linkedin_data.head()

,dataset_id,as_of_date,company_name,followers_count,employees_on_platform,link,industry,date_added,date_updated,description,...,isin,day,year,month,year_month,year_month_01,emp_count_shifted_1,emp_delta,left_per_month,joined_per_month
0,58329,2015-09-14,Goldman Sachs,552254,38124,https://www.linkedin.com/company/1382,Investment Banking,2015-09-14 00:00:00+00,2015-09-14 00:00:00+00,NaN,...,NaN,14,2015,9,2015-09,2015-09-01,0.0,0.0,3285.0,402.0
1,58329,2015-09-15,Goldman Sachs,552862,38141,https://www.linkedin.com/company/1382,Investment Banking,2015-09-15 00:00:00+00,2015-09-15 00:00:00+00,NaN,...,NaN,15,2015,9,2015-09,2015-09-01,38124.0,17.0,3285.0,402.0
2,58329,2015-09-16,Goldman Sachs,553592,38145,https://www.linkedin.com/company/1382,Investment Banking,2015-09-16 00:00:00+00,2015-09-16 00:00:00+00,NaN,...,NaN,16,2015,9,2015-09,2015-09-01,38141.0,4.0,3285.0,402.0
3,58329,2015-09-17,Goldman Sachs,554209,38143,https://www.linkedin.com/company/1382,Investment Banking,2015-09-17 00:00:00+00,2015-09-17 00:00:00+00,NaN,...,NaN,17,2015,9,2015-09,2015-09-01,38145.0,-2.0,3285.0,402.0
4,58329,2015-09-18,Goldman Sachs,554988,38130,https://www.linkedin.com/company/1382,Investment Banking,2015-09-18 00:00:00+00,2015-09-18 00:00:00+00,NaN,...,NaN,18,2015,9,2015-09,2015-09-01,38143.0,-13.0,3285.0,402.0


In [63]:
# Now we need to pick the number of employees at the start and end of month 
## lets pick employees at the beginning of a month-year (first record in groupby) 
## employees at the end of a month-year (last record in groupby)

emp_begin = linkedin_data.groupby(['year_month_01', 'company_name']).first()['employees_on_platform'].reset_index()
emp_begin.rename(columns={'employees_on_platform': 'emp_begin'}, inplace=True)

emp_end = linkedin_data.groupby(['year_month_01', 'company_name']).last()['emp_count_shifted_1'].reset_index()
emp_end.rename(columns={'emp_count_shifted_1': 'emp_end'}, inplace=True)

##merge with main df
linkedin_data = pd.merge(linkedin_data, emp_begin, on=['year_month_01', 'company_name'])
linkedin_data = pd.merge(linkedin_data, emp_end, on=['year_month_01', 'company_name'])

linkedin_data.head()

,dataset_id,as_of_date,company_name,followers_count,employees_on_platform,link,industry,date_added,date_updated,description,...,year,month,year_month,year_month_01,emp_count_shifted_1,emp_delta,left_per_month,joined_per_month,emp_begin,emp_end
0,58329,2015-09-14,Goldman Sachs,552254,38124,https://www.linkedin.com/company/1382,Investment Banking,2015-09-14 00:00:00+00,2015-09-14 00:00:00+00,NaN,...,2015,9,2015-09,2015-09-01,0.0,0.0,3285.0,402.0,38124,35434.0
1,58329,2015-09-15,Goldman Sachs,552862,38141,https://www.linkedin.com/company/1382,Investment Banking,2015-09-15 00:00:00+00,2015-09-15 00:00:00+00,NaN,...,2015,9,2015-09,2015-09-01,38124.0,17.0,3285.0,402.0,38124,35434.0
2,58329,2015-09-16,Goldman Sachs,553592,38145,https://www.linkedin.com/company/1382,Investment Banking,2015-09-16 00:00:00+00,2015-09-16 00:00:00+00,NaN,...,2015,9,2015-09,2015-09-01,38141.0,4.0,3285.0,402.0,38124,35434.0
3,58329,2015-09-17,Goldman Sachs,554209,38143,https://www.linkedin.com/company/1382,Investment Banking,2015-09-17 00:00:00+00,2015-09-17 00:00:00+00,NaN,...,2015,9,2015-09,2015-09-01,38145.0,-2.0,3285.0,402.0,38124,35434.0
4,58329,2015-09-18,Goldman Sachs,554988,38130,https://www.linkedin.com/company/1382,Investment Banking,2015-09-18 00:00:00+00,2015-09-18 00:00:00+00,NaN,...,2015,9,2015-09,2015-09-01,38143.0,-13.0,3285.0,402.0,38124,35434.0


Till this point, new columns have been added in the dataframe that will help in  calculating turnover rate.

In [64]:
#lets save data till this point as pickle
pd.to_pickle(linkedin_data, 'linkedin_data_cleaned.pkl')

In [20]:
# # Let's get count of employees by 'year-month'
# emp_month_avg = linkedin_data.groupby(['year_month_01', 'company_name'])[['employees_on_platform']].sum()

# emp_month_avg.rename(columns={'employees_on_platform': 'emp_month_avg'}, inplace=True)

# ##merge with main df
# linkedin_data = pd.merge(linkedin_data, emp_month_avg, on=['year_month_01', 'company_name'])
# linkedin_data.head()

In [65]:
# lets remove duplicate rows and extra columns and make a new data frame
df_final = linkedin_data[['company_name', 'emp_begin', 'emp_end',
        'year_month_01','left_per_month', 'joined_per_month']]

print(len(df_final))

# drop duplicate rows
df_final = df_final.drop_duplicates()
print(len(df_final))

df_final.head()


2426196
85875


,company_name,emp_begin,emp_end,year_month_01,left_per_month,joined_per_month
0,Goldman Sachs,38124,35434.0,2015-09-01,3285.0,402.0
17,United Technologies,14982,13696.0,2015-09-01,1424.0,168.0
32,Novo Nordisk,26448,26768.0,2015-09-01,0.0,339.0
47,"Lowe's Companies, Inc.",62574,46673.0,2015-09-01,16356.0,534.0
62,UnitedHealth Group,77108,68046.0,2015-09-01,9587.0,582.0


In [66]:
df_final.tail()

,company_name,emp_begin,emp_end,year_month_01,left_per_month,joined_per_month
2426118,EnPro Industries Inc.,231,233.0,2018-07-01,2.0,6.0
2426135,Playa Hotels & Resorts,286,288.0,2018-07-01,1.0,3.0
2426151,Altice USA,4259,4303.0,2018-07-01,7.0,62.0
2426166,Compañía de Minas Buenaventura S.A.A.,1796,1802.0,2018-07-01,4.0,11.0
2426181,KKR & Co. Inc.,2375,2395.0,2018-07-01,0.0,21.0


In [67]:
#find turn-over 
df_final['turn_over'] = (df_final.left_per_month/ (
    (df_final.emp_begin + df_final.emp_end)/2))

df_final['turn_over_p'] = df_final['turn_over'] * 100 

df_final.head()

,company_name,emp_begin,emp_end,year_month_01,left_per_month,joined_per_month,turn_over,turn_over_p
0,Goldman Sachs,38124,35434.0,2015-09-01,3285.0,402.0,0.089317,8.931727
17,United Technologies,14982,13696.0,2015-09-01,1424.0,168.0,0.099310,9.930958
32,Novo Nordisk,26448,26768.0,2015-09-01,0.0,339.0,0.000000,0.000000
47,"Lowe's Companies, Inc.",62574,46673.0,2015-09-01,16356.0,534.0,0.299432,29.943156
62,UnitedHealth Group,77108,68046.0,2015-09-01,9587.0,582.0,0.132094,13.209419


## Plots

In [78]:
from bokeh.plotting import  figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.palettes import Spectral3
from bokeh.io import output_notebook
from bokeh.core.properties import value

In [88]:
# function that'll be used during plotting so that left/join bars can be of different colors
def get_turn_over_data_by_color(df):
    df_left = df[['company_name','year_month_01', 'left_per_month']]
    df_left.rename(columns={'left_per_month':'emp'}, inplace=True)
    df_left['color'] = 'red'
    df_left['legend'] ='left'

    df_joined = df[['company_name', 'year_month_01', 'joined_per_month']]
    df_joined.rename(columns={'joined_per_month':'emp'}, inplace=True) 
    df_joined['color'] = 'blue' 
    df_joined['legend'] ='joined'
    
    df_emp_count= pd.concat([df_joined, df_left], axis=0)

    return df_emp_count

In [89]:
# lets filter the company for which we want to plot data
c_name = 'Apple'
plot_df = df_final[df_final.company_name == c_name]

print(len(plot_df))
plot_df.head()

35


,company_name,emp_begin,emp_end,year_month_01,left_per_month,joined_per_month,turn_over,turn_over_p
407,Apple,103924,93358.0,2015-09-01,11632.0,1203.0,0.117923,11.792257
10830,Apple,93640,108649.0,2015-10-01,8566.0,23566.0,0.084691,8.469071
27371,Apple,108767,111974.0,2015-11-01,0.0,3515.0,0.000000,0.000000
45313,Apple,112213,107968.0,2015-12-01,6249.0,2353.0,0.056762,5.676239
63025,Apple,108173,111114.0,2016-01-01,556.0,3667.0,0.005071,0.507098


### Employee Growth Trend Plot
- x-axis = year_month_01
- y-axis = emp_begin (employee at the beginning of the month)
- y-axis = emp_end (employee at the end of the month)

In [90]:
## bokeh plot
output_notebook()
source = ColumnDataSource(plot_df)

p = figure(x_axis_type='datetime')

p.line(x = 'year_month_01',
       y = 'emp_begin', 
       source= source, 
       legend='# of Employees (Start of month)',
       color= Spectral3[0], 
       line_width=2)

p.line(x = 'year_month_01',
       y = 'emp_end', 
       source= source, 
       legend='# of Employees (End of month)',
       color= 'red', line_width=2)

p.title.text = 'Employee growth trend for '+ c_name
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Number of employees on platform'

hover = HoverTool()
hover.tooltips= [
    ('Employees at the start of month', '@emp_begin'), 
    ('Employees at the end of month', '@emp_end'), 
    ('Month/Year', '@year_month_01 ')
]

p.add_tools(hover)

show(p)

Loading BokehJS ...

### Employee Turnover Plot 
- convert data to stack number of employees left and joined by date and color them so that bars can be of different colors

In [91]:
df_turn_over_plot = get_turn_over_data_by_color(plot_df)
df_turn_over_plot.head()

/Users/adeelahuma/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Users/adeelahuma/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/adeelahuma/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pan

,company_name,year_month_01,emp,color,legend
407,Apple,2015-09-01,1203.0,blue,joined
10830,Apple,2015-10-01,23566.0,blue,joined
27371,Apple,2015-11-01,3515.0,blue,joined
45313,Apple,2015-12-01,2353.0,blue,joined
63025,Apple,2016-01-01,3667.0,blue,joined


In [92]:

## bokeh plot
output_notebook()
source = ColumnDataSource(df_turn_over_plot)

p = figure(x_axis_type='datetime')

p.vbar(x='year_month_01', width=5, 
       top='emp', 
       source=source, 
       color='color', 
       legend='legend')

p.title.text = 'Employee Turn-over for ' + c_name
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Number of employees left or joined'
p.legend?


hover = HoverTool()
hover.tooltips= [
    ('Employees(left/joined)', '@emp'), 
    ('Month/Year', '@year_month_01 ')
]

p.add_tools(hover)

show(p)

Loading BokehJS ...

In [100]:
# plot to show 'left' on negative y-axes

# for left employees multiply with -1 so that they appear on negative y-axis
df_turn_over_plot['modified_emp'] = np.where(df_turn_over_plot.color== 'red', 
                        df_turn_over_plot.emp*-1, 
                        df_turn_over_plot.emp*1)

output_notebook()
source = ColumnDataSource(df_turn_over_plot)

p = figure(x_axis_type='datetime')

p.vbar(x='year_month_01', width=5, 
       top='modified_emp', 
       source=source, 
       color='color', 
       legend='legend')

p.title.text = 'Employee Turn-over for ' + c_name
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Number of employees left or joined'
p.legend?


hover = HoverTool()
hover.tooltips= [
    ('Employees(left/joined)', '@emp'), 
    ('Month/Year', '@year_month_01 ')
]

p.add_tools(hover)

show(p)

Loading BokehJS ...

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: modified_emp [renderer: GlyphRenderer(id='f5b4088b-2939-4181-b77e-f14b0792fd5b', ...)]


## Final Data 
- save final data as a pickle so that application can use it for interactive plots

In [101]:
pd.to_pickle(df_final, 'application_data.pickle')

In [102]:
#unique companies in final data
len(df_final.company_name.unique())

5028

In [103]:
len(df_final)

85875